In [ ]:
import pandas as pd
import sqlalchemy
from config import configcovid

In [ ]:
# Download the data from https://covid.ourworldindata.org/ size around 80 MB
df = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')

# Save to local drive in data folder
df.to_csv('data/covid_data.csv')

# Read from local drive
# df = pd.read_csv('data/covid_data.csv')

In [ ]:
# Define column for each table
str_col = configcovid.str_col
date_col = configcovid.date_col
float_col = configcovid.float_col
dimention_col = configcovid.dimention_col
covid_col = configcovid.covid_col
vaccinated_col = configcovid.vaccinated_col
excess_col = configcovid.excess_col

# Change column data type
df[str_col] = df[str_col].astype('string')
df[float_col] = df[float_col].astype('float')
for col in date_col:
    df[col] = pd.to_datetime(df[col])

In [4]:
# Split data into different dataframe 
df_dimention = df[list(configcovid.dimention_col.keys())].drop_duplicates().reset_index(drop=True)
df_covid = df[list(configcovid.covid_col.keys())]
df_vaccinated = df[list(configcovid.vaccinated_col.keys())]
df_excess = df[list(configcovid.excess_col.keys())].query("not(excess_mortality_cumulative_absolute.isnull())").reset_index(drop=True)

In [5]:
def sql_insert(data=pd.DataFrame,name=str,engine=sqlalchemy.engine,schema =str,col_type=dict)->None:
    print(f'Inserting data for {name}')
    try:
        data.to_sql(con=engine,schema=schema,name=name,index=False,if_exists='replace',dtype=col_type)
    except Exception as err:
        print(f'An error occur {err} table.')

    print(f'Finish inserting data')
    print('-----------------------')
    return None

def sql_excute(query=str,engine=sqlalchemy.engine)->None:
    print('Executing query command...')
    engine.execute(statement=query)
    print('Finish Executing')
    print('-----------------------')
    return None

def sql_excute_sp(query=str,session=sqlalchemy.engine.cursor)->None:
    print('Executing Store Procedure...')
    session.execute(query)
    session.commit()
    print('Finish Executing')
    print('-----------------------')
    return None

In [6]:
# Connect to local MS SQL Server with fast excute on
eng = sqlalchemy.create_engine("mssql://localhost/Covid_Database?driver=ODBC Driver 17 for SQL Server",fast_executemany=True)
schema = 'stg'
schema_query = f"""
IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = '{schema}')
BEGIN
EXEC('CREATE SCHEMA {schema}')
END"""

# Create stg schema if not exist
sql_excute(schema_query,eng)

# Write data into sql server
df_insert = [
    (df_dimention,'stg_dimention',dimention_col),
    (df_covid,'stg_covid',covid_col),
    (df_vaccinated,'stg_vaccinated',vaccinated_col),
    (df_excess,'stg_excess',excess_col)
    ]
for df,name,col_type in df_insert:
    sql_insert(data=df,name=name,engine=eng,schema=schema,col_type=col_type)

# Create insert sp
for sp in configcovid.sp_list:
    sql_excute(query=sp,engine=eng)

# Excute the sp to insert the data if first time will create table if not will insert date more than the current data
session = eng.raw_connection().cursor()
for sp in configcovid.sp_excute_list:
    query = f'exec [dbo].[{sp}]'
    print(sp)
    sql_excute_sp(query=query,session=session)
session.close()

Executing query command...
Finish Executing
-----------------------
Inserting data for stg_dimention
Finish inserting data
-----------------------
Inserting data for stg_covid
Finish inserting data
-----------------------
Inserting data for stg_vaccinated
Finish inserting data
-----------------------
Inserting data for stg_excess
Finish inserting data
-----------------------
Executing query command...
Finish Executing
-----------------------
Executing query command...
Finish Executing
-----------------------
Executing query command...
Finish Executing
-----------------------
Executing query command...
Finish Executing
-----------------------
Stg_Covid_insert
Executing Store Procedure...
Finish Executing
-----------------------
Stg_Vaccinated_insert
Executing Store Procedure...
Finish Executing
-----------------------
Stg_Dimention_insert
Executing Store Procedure...
Finish Executing
-----------------------
Stg_Excess_insert
Executing Store Procedure...
Finish Executing
----------------